In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from datasets import load_dataset,Dataset,get_dataset_config_names
import warnings # to ignore warnings
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
pretraining_language = 'ar'

In [3]:
squad = load_dataset("squad")

Reusing dataset squad (C:\Users\Subha\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 153.83it/s]


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# squad.set_format("pandas")
# df = squad["train"][:]
# df.head()

In [7]:
# df["answer_text"] = df.answers.apply(lambda x: x["text"])
# df["answer_start"] = df.answers.apply(lambda x: x["answer_start"])

In [8]:
# def preprocess_answers(answer_info):
#     ans_start_index = answer_info[0]
#     ans_text = answer_info[1]
#     return {
#         'answer_start': [int(ans_start_index)],
#         'text': [ans_text]
#     }

In [9]:
# df = df.sample(frac=1, random_state=42)
# df['answers'] = df[['answer_start', 'answer_text']].apply(preprocess_answers, axis=1)

In [10]:
# df.head()

In [11]:
# for col_name in df.columns:
#     if col_name in ['id', 'context', 'question', 'answer_text']:
#         df[col_name] = df[col_name].astype(str)
#     elif col_name in ['answer_start']:
#         df[col_name] = df[col_name].astype('int64')
#     print(df[col_name].dtype)

In [12]:
# squad_train_ds = Dataset.from_pandas(df)

In [13]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# model = AutoModelForQuestionAnswering.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")
model = AutoModelForQuestionAnswering.from_pretrained("subhasisj/Ar-Mulitlingula-MiniLM")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Downloading: 100%|██████████| 695/695 [00:00<00:00, 348kB/s]
Downloading: 100%|██████████| 450M/450M [00:28<00:00, 16.5MB/s]   
Some weights of the model checkpoint at subhasisj/Ar-Mulitlingula-MiniLM were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [15]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [16]:
batch_size = 8

In [17]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\Subha\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-339885a2ba07a6ff.arrow
100%|██████████| 11/11 [00:05<00:00,  2.01ba/s]


In [18]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    f"{pretraining_language}-pretrained-squad-qa-minilmv2-{batch_size}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 1.5e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 5,
    fp16=True,
    weight_decay = 0.01,
    push_to_hub=True
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad["train"],
    eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/subhasisj/ar-pretrained-squad-qa-minilmv2-8 into local empty directory.
Using amp half precision backend


In [21]:
trainer.train()

***** Running training *****
  Num examples = 89597
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 7000
  7%|▋         | 500/7000 [11:35<2:33:30,  1.42s/it]

{'loss': 4.8525, 'learning_rate': 1.0714285714285714e-05, 'epoch': 0.36}


 14%|█▍        | 1000/7000 [22:36<2:09:41,  1.30s/it]

{'loss': 3.2915, 'learning_rate': 1.4285714285714285e-05, 'epoch': 0.71}


 20%|██        | 1400/7000 [31:11<2:02:00,  1.31s/it]***** Running Evaluation *****
  Num examples = 10918
  Batch size = 8
                                                     
 20%|██        | 1400/7000 [32:18<2:02:00,  1.31s/it]Saving model checkpoint to ar-pretrained-squad-qa-minilmv2-8\checkpoint-1400
Configuration saved in ar-pretrained-squad-qa-minilmv2-8\checkpoint-1400\config.json


...

                                                     
100%|██████████| 7000/7000 [2:44:51<00:00,  1.28s/it]Saving model checkpoint to ar-pretrained-squad-qa-minilmv2-8\checkpoint-7000
Configuration saved in ar-pretrained-squad-qa-minilmv2-8\checkpoint-7000\config.json


{'eval_loss': 1.1237053871154785, 'eval_runtime': 66.6514, 'eval_samples_per_second': 163.807, 'eval_steps_per_second': 20.48, 'epoch': 5.0}


Model weights saved in ar-pretrained-squad-qa-minilmv2-8\checkpoint-7000\pytorch_model.bin
tokenizer config file saved in ar-pretrained-squad-qa-minilmv2-8\checkpoint-7000\tokenizer_config.json
Special tokens file saved in ar-pretrained-squad-qa-minilmv2-8\checkpoint-7000\special_tokens_map.json
tokenizer config file saved in ar-pretrained-squad-qa-minilmv2-8\tokenizer_config.json
Special tokens file saved in ar-pretrained-squad-qa-minilmv2-8\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 7000/7000 [2:45:51<00:00,  1.42s/it]

{'train_runtime': 9951.9396, 'train_samples_per_second': 45.015, 'train_steps_per_second': 0.703, 'train_loss': 1.7748914271763392, 'epoch': 5.0}


TrainOutput(global_step=7000, training_loss=1.7748914271763392, metrics={'train_runtime': 9951.9396, 'train_samples_per_second': 45.015, 'train_steps_per_second': 0.703, 'train_loss': 1.7748914271763392, 'epoch': 5.0})

In [22]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [23]:
squad_valid = load_dataset("squad",split="validation")

Reusing dataset squad (C:\Users\Subha\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [24]:
validation_features = squad_valid.map(
    prepare_validation_features,
    batched=True,
    remove_columns=squad_valid.column_names
)

100%|██████████| 11/11 [00:49<00:00,  4.54s/ba]


In [25]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10918
  Batch size = 8
100%|█████████▉| 1364/1365 [01:14<00:00, 18.25it/s]

In [26]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [27]:
max_answer_length = 30

In [28]:
import collections

examples = squad_valid
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

100%|██████████| 1365/1365 [01:24<00:00, 18.25it/s]

In [29]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [30]:
final_predictions = postprocess_qa_predictions(squad_valid, validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10918 features.


100%|██████████| 10570/10570 [00:42<00:00, 250.95it/s]


In [31]:
from datasets import load_metric
metric = load_metric("squad")


In [32]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad_valid]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 76.50898770104068, 'f1': 84.64469654388077}

https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb